In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from porespy import generators
from tomo_encoders.misc_utils.feature_maps_vis import view_midplanes
import cupy as cp
import time
import h5py
from tomopy import project
sys.path.append('/data02/MyArchive/aisteer_3Dencoders/TomoEncoders/tomo_encoders/tasks/sparse_segmenter/')
from recon_subvol import fbp_filter, test_recon_patch, recon_patches_3d
from tomo_encoders import DataFile, Patches
import os
import signal
import tomocg as pt

In [2]:
# # Use managed memory
# cp.cuda.set_allocator(cp.cuda.MemoryPool(cp.cuda.malloc_managed).malloc)

### Read projections data  

In [3]:
filename = 'mli_L206_HT_650_L3'
projs_path = '/data02/MyArchive/AM_part_Xuan/projs' 
binning = 1 
ntheta = 3000

In [4]:
read_fpath = os.path.join(projs_path, filename + '_projs_bin%i_ntheta%i.hdf5'%(binning,ntheta))
with h5py.File(read_fpath, 'r') as hf:
    projs = np.asarray(hf['data'][:])
    theta = np.asarray(hf['theta'][:])
    center = float(np.asarray(hf['center'][()]))

## Reconstruct one full slice  

In [6]:
vol_shape = projs.shape[1:] + (projs.shape[-1],)
vol_shape

(1102, 4200, 4200)

In [7]:
model_size = (64,64,64)
kwargs = {"initialize_by" : 'regular-grid',
          "patch_size" : model_size, \
          "stride" : 1}
p_grid = Patches(vol_shape, **kwargs)
print(p_grid.points.shape)
p_grid._check_valid_points()

(71825, 3)


In [8]:
p3d = p_grid.select_random_sample(100)

In [10]:
print(p3d.widths[:1], p3d.points[:1])

[[64 64 64]] [[ 896 3264 1408]]


In [11]:
with cp.cuda.Device(0):
    center = projs.shape[-1]//2.0
    sub_vols = recon_patches_3d(projs, theta, center, p3d, mem_limit_gpu = 40.0, apply_fbp = True)
    sub_vol = 1.0*(sub_vol - sub_vol.min()) / (sub_vol.max() - sub_vol.min())

ValueError: mem limit breached

In [ ]:
center = projs.shape[-1]//2.0


# point = [500//binning, 0, 0]
# point = [0, 0, 0]
# width = [1, 4200//binning, 4200//binning]


# z, y, x
point = [500, 2000//binning, 2000//binning]
width = [128, 256, 256]

# point = [500, 0, 0]
# width = [1, 2100, 2100]


with cp.cuda.Device(0):
    center = projs.shape[-1]//2.0
    sub_vol = recon_patch(projs, theta, center, point, width, \
                          mem_limit_gpu = 40.0,
                          apply_fbp=True, nzc = 8)
    sub_vol = 1.0*(sub_vol - sub_vol.min()) / (sub_vol.max() - sub_vol.min())

In [ ]:
5%2

In [ ]:
sub_vol.shape

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (16,16))
ax.imshow(sub_vol[0], cmap = 'gray')